# E3: GDP per capita performance index
## - Mimic SUM() OVER() window function in PostgreSQL

In [1]:
import pandas as pd
import numpy as np

In [2]:
summer_games = pd.read_csv("summer_games.csv")
winter_games = pd.read_csv("winter_games.csv")
countries = pd.read_csv("countries.csv")
athletes = pd.read_csv("athletes.csv")
country_stats = pd.read_csv("country_stats.csv")

In [3]:
display(summer_games.head(2))
display(winter_games.head(2))
display(countries.head(2))
display(athletes.head(2))
display(country_stats.head(2))

,sport,event,year,athlete_id,country_id,bronze,silver,gold
0,Gymnastics,Gymnastics Men's Individual All-Around,2016-01-01,51,173,NaN,NaN,NaN
1,Gymnastics,Gymnastics Men's Floor Exercise,2016-01-01,51,173,NaN,NaN,NaN


,sport,event,year,athlete_id,country_id,bronze,silver,gold
0,Alpine Skiing,Alpine Skiing Women's Slalom,2014-01-01,126,89,NaN,NaN,NaN
1,Alpine Skiing,Alpine Skiing Women's Super G,2014-01-01,463,102,NaN,NaN,NaN


,id,country,region
0,1,AFG - Afghanistan,ASIA (EX. NEAR EAST)
1,2,ALB - Albania,EASTERN EUROPE


,id,name,gender,age,height,weight
0,51,Nstor Abad Sanjun,M,23,167.0,64.0
1,55,Antonio Abadia Beci,M,26,170.0,65.0


,year,country_id,gdp,population,nobel_prize_winners
0,2000-01-01,1,NaN,20093756.0,NaN
1,2000-01-01,2,3.632044e+09,3089027.0,NaN


In [4]:
country_stats['year'] = pd.to_datetime(country_stats['year'])

In [5]:
region_country_2016_gdp = countries.merge(country_stats, left_on='id', right_on='country_id', how='inner')\
         .loc[lambda df: (df['year'] == '2016') & (~df['gdp'].isna())]\
         .assign(gdp_per_million = lambda df: df['gdp'].div(df['population']))\
         .assign(gdp_per_million_total = lambda df: df['gdp'].sum()/(df['population'].sum()))

In [6]:
performance_index_df = \
region_country_2016_gdp.groupby(['region', 'country'])[['gdp_per_million', 'gdp_per_million_total']]\
                       .agg({'gdp_per_million':'sum', 'gdp_per_million_total':'mean'}).reset_index()\
                       .sort_values('gdp_per_million', ascending=False)\
                       .assign(performance_index = lambda df: df['gdp_per_million'].div(df['gdp_per_million_total']))

In [7]:
performance_index_df

,region,country,gdp_per_million,gdp_per_million_total,performance_index
178,WESTERN EUROPE,LUX - Luxembourg,100738.684222,10213.471203,9.863315
184,WESTERN EUROPE,SUI - Switzerland,79865.998482,10213.471203,7.819672
181,WESTERN EUROPE,NOR - Norway,70889.990083,10213.471203,6.940832
175,WESTERN EUROPE,IRL - Ireland,64100.426153,10213.471203,6.276067
176,WESTERN EUROPE,ISL - Iceland,60529.926756,10213.471203,5.926479
...,...,...,...,...,...
144,SUB-SAHARAN AFRICA,MOZ - Mozambique,382.069457,10213.471203,0.037408
149,SUB-SAHARAN AFRICA,NIG - Niger,367.955986,10213.471203,0.036027
142,SUB-SAHARAN AFRICA,MAW - Malawi,300.307665,10213.471203,0.029403
118,SUB-SAHARAN AFRICA,BDI - Burundi,285.727442,10213.471203,0.027976
